# Scraping Apartment Listings for my Apartment Hunt

This is an attempt for me to learn how to web scrape, clean my own data, and generate a predictive model of apartments that selects apartments that I would like based on price, size, etc.

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

## Initial analysis was done on the df variable, but I intend to apply cleanup process on others

In [6]:
df = pd.read_csv("San_Diego.csv")

sd = pd.read_csv("San_Diego.csv")
oc = pd.read_csv("Orange_County.csv")
la = pd.read_csv("Los_Angeles.csv")

In [7]:
df.columns

Index(['Unnamed: 0', 'Address', 'Beds', 'Phone', 'Price', 'Title'], dtype='object')

Very minimal dataset, but as I improve my cleanup and analysis skills, I will scrape more content and perform more relevant transformations.

Proof of concept to start.

In [8]:
# set variable price to dataframe column 'Price', which is just a pandas series
price = df["Price"]
price

0       $1,760 - 2,040
1       $1,765 - 2,135
2       $1,630 - 2,000
3       $1,600 - 2,050
4       $1,970 - 3,490
5      $2,448 - 12,793
6       $2,644 - 6,892
7       $1,913 - 4,966
8       $2,102 - 6,860
9       $1,733 - 3,595
10      $2,300 - 6,070
11      $1,955 - 6,135
12      $2,060 - 4,815
13      $1,975 - 6,770
14      $2,210 - 6,912
15      $1,910 - 3,757
16      $2,075 - 4,425
17      $2,149 - 5,205
18      $1,770 - 3,243
19      $1,453 - 2,503
20      $2,425 - 4,350
21      $1,955 - 3,750
22     $2,051 - 20,000
23      $2,010 - 6,180
24      $2,039 - 3,472
25      $1,849 - 3,424
26      $1,925 - 5,145
27     $1,700 - 10,000
28      $1,480 - 3,074
29      $1,852 - 2,991
            ...       
222     $2,220 - 4,855
223     $1,861 - 2,148
224             $2,245
225             $1,660
226     $1,330 - 2,341
227     $1,643 - 2,606
228     $1,897 - 3,714
229     $1,784 - 2,874
230     $1,645 - 2,085
231     $1,762 - 2,556
232     $1,618 - 2,520
233     $2,395 - 3,895
234     $1,

In [9]:
# this is indeed a string, I want it to be an int but i need to transform the data first
type(df["Price"][0])

str

In [10]:
# Start by removing the $ symbol
df["Price"] = df["Price"].str[1:]
price

0       1,760 - 2,040
1       1,765 - 2,135
2       1,630 - 2,000
3       1,600 - 2,050
4       1,970 - 3,490
5      2,448 - 12,793
6       2,644 - 6,892
7       1,913 - 4,966
8       2,102 - 6,860
9       1,733 - 3,595
10      2,300 - 6,070
11      1,955 - 6,135
12      2,060 - 4,815
13      1,975 - 6,770
14      2,210 - 6,912
15      1,910 - 3,757
16      2,075 - 4,425
17      2,149 - 5,205
18      1,770 - 3,243
19      1,453 - 2,503
20      2,425 - 4,350
21      1,955 - 3,750
22     2,051 - 20,000
23      2,010 - 6,180
24      2,039 - 3,472
25      1,849 - 3,424
26      1,925 - 5,145
27     1,700 - 10,000
28      1,480 - 3,074
29      1,852 - 2,991
            ...      
222     2,220 - 4,855
223     1,861 - 2,148
224             2,245
225             1,660
226     1,330 - 2,341
227     1,643 - 2,606
228     1,897 - 3,714
229     1,784 - 2,874
230     1,645 - 2,085
231     1,762 - 2,556
232     1,618 - 2,520
233     2,395 - 3,895
234     1,873 - 2,889
235     1,739 - 2,345
236     1,

In [11]:
# Next, remove the comma
price = price.str.replace(',','')
price

0       1760 - 2040
1       1765 - 2135
2       1630 - 2000
3       1600 - 2050
4       1970 - 3490
5      2448 - 12793
6       2644 - 6892
7       1913 - 4966
8       2102 - 6860
9       1733 - 3595
10      2300 - 6070
11      1955 - 6135
12      2060 - 4815
13      1975 - 6770
14      2210 - 6912
15      1910 - 3757
16      2075 - 4425
17      2149 - 5205
18      1770 - 3243
19      1453 - 2503
20      2425 - 4350
21      1955 - 3750
22     2051 - 20000
23      2010 - 6180
24      2039 - 3472
25      1849 - 3424
26      1925 - 5145
27     1700 - 10000
28      1480 - 3074
29      1852 - 2991
           ...     
222     2220 - 4855
223     1861 - 2148
224            2245
225            1660
226     1330 - 2341
227     1643 - 2606
228     1897 - 3714
229     1784 - 2874
230     1645 - 2085
231     1762 - 2556
232     1618 - 2520
233     2395 - 3895
234     1873 - 2889
235     1739 - 2345
236     1550 - 2350
237     3490 - 4950
238     1545 - 1860
239     1635 - 2500
240     2050 - 2150


In [12]:
# Lastly, let's split the two values into separate columns...a min_price and max_price column
price = price.str.split("-", expand=True)
price

,0,1
0,1760,2040
1,1765,2135
2,1630,2000
3,1600,2050
4,1970,3490
5,2448,12793
6,2644,6892
7,1913,4966
8,2102,6860
9,1733,3595


Note, there are some values that do not have a range, so some values have 'none'

We'll deal with that in a bit.

In [13]:
# verifying that the values are indeed still strings
type(price[1][0])

str

In [14]:
# Let's rename the columns
price.rename(index=str, columns={0: 'min_price', 1: 'max_price'}, inplace=True)

In [15]:
# And notice that the individual string has whitespace...let's remove
price['min_price'][0]

'1760 '

In [16]:
# We can apply a lambda function to remove whitespace (strip)
price_trimmed = price.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [17]:
# Let's check
price_trimmed['min_price'][0]

'1760'

Cool, so we have trimmed the values of white space, so we have transformed the data to a point where we can attempt to convert to an int

In [18]:
price_trimmed

,min_price,max_price
0,1760,2040
1,1765,2135
2,1630,2000
3,1600,2050
4,1970,3490
5,2448,12793
6,2644,6892
7,1913,4966
8,2102,6860
9,1733,3595


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 6 columns):
Unnamed: 0    252 non-null int64
Address       252 non-null object
Beds          252 non-null object
Phone         252 non-null object
Price         252 non-null object
Title         252 non-null object
dtypes: int64(1), object(5)
memory usage: 11.9+ KB


A slight tangent, but I want to concat the df and price_trimmed dataframes

In [20]:
len(df)

252

In [21]:
len(price_trimmed)

252

Something I had struggled with for a while was that, despite the fact that the length of each dataframe is the same...

In [22]:
df.index

RangeIndex(start=0, stop=252, step=1)

In [23]:
price_trimmed.index

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '242', '243', '244', '245', '246', '247', '248', '249', '250', '251'],
      dtype='object', length=252)

The index for each differed in some way. This led to incompatibilities in the concatenation.

To address this issue, I decided to reset each index.

In [24]:
test_df = df.reset_index()

In [25]:
test_df.index

RangeIndex(start=0, stop=252, step=1)

In [26]:
test_price = price_trimmed.reset_index()

In [27]:
test_price.index

RangeIndex(start=0, stop=252, step=1)

So notice now that the index for both dataframes are the same, so I should be able to concat into one dataframe:

new combined dataframe is df1.

In [28]:
df1 = pd.concat([test_df, test_price], axis=1)

In [30]:
df1.head()

,index,Unnamed: 0,Address,Beds,Phone,Price,Title,index,min_price,max_price
0,0,0,"6304 Rancho Mission Rd, San Diego, CA 92108",1-2 Bed,844-873-6008,"1,760 - 2,040",Prado Apartments,0,1760,2040
1,1,1,"6363 Beadnell Way, San Diego, CA 92117",1-2 Bed,844-220-7316,"1,765 - 2,135",Barclay Square,1,1765,2135
2,2,2,"3454 Ruffin Rd, San Diego, CA 92123",1-2 Bed,844-460-0665,"1,630 - 2,000",Village Glen,2,1630,2000
3,3,3,"3625 Earnscliff Pl, San Diego, CA 92111",1-2 Bed,844-849-4236,"1,600 - 2,050",Genesee Park ** MOVE IN SPECIALS **,3,1600,2050
4,4,4,"1281 9th Ave, San Diego, CA 92101",1-2 Bed,619-894-8653,"1,970 - 3,490",Vantage Pointe,4,1970,3490


They are combined to one table, but we also have some weird columns that are worth removing:

In [31]:
# let's remove columns titled index...it's redundant since have have an actual index to work with
df1 = df1.drop(columns=['index'], axis=1)

In [32]:
# also want to remove this weird column named Unnamed: 0
df1 = df1.drop(columns=['Unnamed: 0'])

In [33]:
# now let's take a look at the table head
df1.head()

,Address,Beds,Phone,Price,Title,min_price,max_price
0,"6304 Rancho Mission Rd, San Diego, CA 92108",1-2 Bed,844-873-6008,"1,760 - 2,040",Prado Apartments,1760,2040
1,"6363 Beadnell Way, San Diego, CA 92117",1-2 Bed,844-220-7316,"1,765 - 2,135",Barclay Square,1765,2135
2,"3454 Ruffin Rd, San Diego, CA 92123",1-2 Bed,844-460-0665,"1,630 - 2,000",Village Glen,1630,2000
3,"3625 Earnscliff Pl, San Diego, CA 92111",1-2 Bed,844-849-4236,"1,600 - 2,050",Genesee Park ** MOVE IN SPECIALS **,1600,2050
4,"1281 9th Ave, San Diego, CA 92101",1-2 Bed,619-894-8653,"1,970 - 3,490",Vantage Pointe,1970,3490


Nice! Making some progress! Let's sort by the min_price now.

In [34]:
# sort by min values
df_sort = df1.sort_values(by=['min_price'])

In [35]:
df_sort.head()

,Address,Beds,Phone,Price,Title,min_price,max_price
102,"819 University Ave, San Diego, CA 92103",Studio,619-333-2836,"1,000 - 1,270",Vantaggio Suites Hillcrest,1000,1270
249,"5360 Marengo Ave, La Mesa, CA 91942",Studio - 2 Bed,844-258-2361,"1,025 - 1,500",Patrician Apartments,1025,1500
43,"1736 State St, San Diego, CA 92101",Studio,619-317-1623,"1,075 - 1,340",Vantaggio Suites at San Diego,1075,1340
243,"240 Quintard St, Chula Vista, CA 91911",1-3 Bed,844-423-1798,"1,100 - 1,500",Casa Madrid Apartments,1100,1500
244,"1225 Broadway, Chula Vista, CA 91911",1-2 Bed,844-210-2387,"1,100 - 1,500",Sunset Villa Apartments,1100,1500


And then reset the index.

In [36]:
# reset index
df_sort = df_sort.reset_index()

In [37]:
df_sort.head()

,index,Address,Beds,Phone,Price,Title,min_price,max_price
0,102,"819 University Ave, San Diego, CA 92103",Studio,619-333-2836,"1,000 - 1,270",Vantaggio Suites Hillcrest,1000,1270
1,249,"5360 Marengo Ave, La Mesa, CA 91942",Studio - 2 Bed,844-258-2361,"1,025 - 1,500",Patrician Apartments,1025,1500
2,43,"1736 State St, San Diego, CA 92101",Studio,619-317-1623,"1,075 - 1,340",Vantaggio Suites at San Diego,1075,1340
3,243,"240 Quintard St, Chula Vista, CA 91911",1-3 Bed,844-423-1798,"1,100 - 1,500",Casa Madrid Apartments,1100,1500
4,244,"1225 Broadway, Chula Vista, CA 91911",1-2 Bed,844-210-2387,"1,100 - 1,500",Sunset Villa Apartments,1100,1500


In [39]:
df_sort['min_price'].astype(float)

ValueError: could not convert string to float: 'all for Rent'

Uh oh, not every entry can be easily converted to a float. Seems we have columns with strings rather than prices.

## TODO: remove entries with random strings
## TODO: verify entries can be converted to int or float (numeric value)
## TODO: create a new column of the average price (average of the low and high prices)
## TODO: group 'Beds' based on studio, 1 bed, 2 bed, etc.
## TODO: scrape more data and provide actual geolocation

# This is not a static notebook, I will make several changes. This is just a preview of my data exploration and cleaning.